In [1]:
# 필수 라이브러리 호출
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score as acc_sc

In [2]:
# 데이터 로드
st_default_org = pd.read_csv('./st_default.csv')
st_default_org

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.62507
1,2,No,Yes,817.180407,12106.13470
2,3,No,No,1073.549164,31767.13895
3,4,No,No,529.250605,35704.49394
4,5,No,No,785.655883,38463.49588
...,...,...,...,...,...
9995,9996,No,No,711.555020,52992.37891
9996,9997,No,No,757.962918,19660.72177
9997,9998,No,No,845.411989,58636.15698
9998,9999,No,No,1569.009053,36669.11236


In [5]:
# 식별자 제거: Unnamed: 0 제거
st_default_df = st_default_org.iloc[:, 1:]  # 위치기반 인덱싱
st_default_df  # 전체 데이터 (X + Y)

,default,student,balance,income
0,No,No,729.526495,44361.62507
1,No,Yes,817.180407,12106.13470
2,No,No,1073.549164,31767.13895
3,No,No,529.250605,35704.49394
4,No,No,785.655883,38463.49588
...,...,...,...,...
9995,No,No,711.555020,52992.37891
9996,No,No,757.962918,19660.72177
9997,No,No,845.411989,58636.15698
9998,No,No,1569.009053,36669.11236


In [7]:
# 전체 데이터에서 X와 Y 데이터 쪼깨기
# X_data와 y_target 정의

# X_data 정의
X_data = st_default_df.drop('default', axis=1)  # st_default_df.iloc[:,1:]
X_data

,student,balance,income
0,No,729.526495,44361.62507
1,Yes,817.180407,12106.13470
2,No,1073.549164,31767.13895
3,No,529.250605,35704.49394
4,No,785.655883,38463.49588
...,...,...,...
9995,No,711.555020,52992.37891
9996,No,757.962918,19660.72177
9997,No,845.411989,58636.15698
9998,No,1569.009053,36669.11236


In [8]:
# y_target 정의
y_target = st_default_df.default
y_target

0       No
1       No
2       No
3       No
4       No
        ..
9995    No
9996    No
9997    No
9998    No
9999    No
Name: default, Length: 10000, dtype: object

### EDA를 통한 데이터 확인

In [9]:
X_data.info()  # 'student' 변수가 'object'임 --> 전처리 필요!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   student  10000 non-null  object 
 1   balance  10000 non-null  float64
 2   income   10000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


In [10]:
X_data.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462905
50%,823.636973,34552.644800
75%,1166.308387,43807.729275
max,2654.322576,73554.233500


In [11]:
# 크기 확인
print('X_data의 형태:', X_data.shape)
print('y_target의 형태:', y_target.shape)

X_data의 형태: (10000, 3)
y_target의 형태: (10000,)


In [15]:
## 인코딩
# '문자 -> 숫자' 변환
# 만약, 문자 col. 제거 가능? OK --> but 없애려는 명확한 이유가 있어야 함.
# 그렇다면, 'Student'값은 one-hot encoding, 'Default'값은 label encoding

X_data_df = pd.get_dummies(X_data)  # one-hot encoding :: ndarray에서는 문자 -> 레이블 -> 원핫 순서로 가는데, pandas에서는 한번에 해줌. 

y_target[:5] # Series

0    No
1    No
2    No
3    No
4    No
Name: default, dtype: object

In [14]:
# y값에는 Label Encoding(레이블 인코딩)
from sklearn.preprocessing import LabelEncoder

le_encode = LabelEncoder()
y_target_scaled = le_encode.fit_transform(y_target)  # scaling 완료

# y_target 값 확인
# y_target_scaled.value_counts()  # error :: numpy에는 value_counts()가 없음. pandas여야 함. 따라서 Series로 바꿔주어야 함.
pd.Series(y_target_scaled).value_counts()
# 결과값을 보니 0과 1의 관측수가 불균형적임 ==> imbalanced_data !!

0    9667
1     333
dtype: int64

In [16]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled)  # stratify=y_target_scaled :: y_target_scaled의 imbalanced 한 것을 고려해서 층화해달라!

In [19]:
# 의사결정나무 적용
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)  # 학습 수행

# 예측
y_pred = dt_clf.predict(X_test)

In [21]:
# 평가지표
print(f'정확도: {acc_sc(y_pred, y_test)}')

정확도: 0.9528


In [26]:
# CV - 교차검증 수행 :: cross_val_score
from sklearn.model_selection import cross_val_score

# 보통 전체 데이터에서 얼마나 CV의 정확도가 나오는 지 본다.
scores = cross_val_score(dt_clf, X_train, y_train, scoring='accuracy', cv=10)  # 'scoring=' 평가지표를 무엇으로 할 것인가?

print(f'교차 검증별 정확도: {np.round(scores, 4)}')
print(f'평균 검증별 정확도: {np.round(np.mean(scores), 4)}')
# X_train, y_train의 교차검증을 확인해보니 위에서 본 평가지표(0.9528)와 비슷한 레벨이므로, 오버피팅이라 보기는 힘들고 정확하게 예측이 되었다고 볼 수 있다.

교차 검증별 정확도: [0.9467 0.9533 0.948  0.96   0.968  0.9627 0.9573 0.9587 0.952  0.9573]
평균 검증별 정확도: 0.9564


In [27]:
## Think About Under Questions
y_target.value_counts()  # 'No': 9667, 'Yes': 333

# 1. 왜 데이터의 해당하는 정확도 값이 높게 나왔을까? - 'No'와 'Yes'의 관측수가 imbalanced 한 특징을 고려해서 stratified 를 진행하였기 때문
# 2. 데이터 특징 - imbalanced data
# 3. 데이터 문제점 - 불균형적인 데이터로 인해 오버피팅될 수 있는 문제점 존재
# 4. 해결방안
    # [1] oversampling vs undersampling - (ex. 둘 중 적은 관측수를 가진 'Yes: 333'에 맞춰서 'No'관측수를 333만 뽑기 :: undersampling)
    # [2] Data augentation - 이미지 분석
    # [3] get more data - 데이터 더 구하기
    # [4] change our metrics - 다른 평가지표 가져와서 쓰기 (ex. precision/recall/f1-score/roc_auc ..)

No     9667
Yes     333
Name: default, dtype: int64